<a href="https://colab.research.google.com/github/shoma3571/recommend-engine/blob/main/chapter2/2_MatrixFactorization_MovieLens100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

各ユーザーにおすすめアイテムトップ10を計算する。
パラメータとしてここでの潜在因子 (行列分解した時のベクトルの長さ)の次元を50とする

In [2]:
topk = 10
rank_list = [i+1 for i in range(topk)]
latent = 50

In [3]:
ML100K_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
dataset = pd.read_csv(
    ML100K_URL,
    names = ["user_id", "item_id", "rating", "timestamp"],
    sep = "\t"
)

matrix_data = csr_matrix(
    (dataset.rating,
    (dataset.user_id, dataset.item_id))
)

非負値行列因子分解 (Non-negative Matrix Factorization: NMF) はある行列XをW, Hに近似的に分解する方法の1つ。すべての値が非負となるように学習・予測していくモデル。

In [4]:
nmf = NMF(n_components=latent)
# ユーザー因子行列
W = nmf.fit_transform(matrix_data)
# アイテム因子行列
H = nmf.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


In [5]:
# ユーザー因子行列とアイテム因子行列を掛け合わせ、評価行列を復元する
WH = np.dot(W, H)

# すでに評価したユーザー・アイテムの組み合わせを取り除く
recommend_matrix = np.where(
    matrix_data.toarray(), 0, WH
)

ユーザーから見たそれぞれのアイテムの評価トップ10を絞り込むことで推薦データが完成する

In [6]:
# ユニークなユーザーIDのリストを作成する
uq_users = np.sort(dataset.user_id.unique().tolist())

# 推薦結果を格納するテーブルを作成する
df_recommend_list = pd.DataFrame(
    columns = ['user_id', 'item_id', 'score', 'rank']
)

# 各ユーザーに対してトップ10アイテムを絞り込む
for user_id in uq_users:
  # 対象ユーザーの未接触アイテムへのスコア情報を取得
  item_scores = recommend_matrix[user_id]

  df_recommend = pd.DataFrame()
  df_recommend['user_id'] = [user_id] * topk
  # トップ10アイテムのアイテムIDを取得
  df_recommend['item_id'] = np.argsort(item_scores)[::-1][:topk]
  # トップ10アイテムのスコアを取得
  df_recommend['score'] = np.sort(item_scores)[::-1][:topk]
  df_recommend['rank'] = rank_list

  df_recommend_list = df_recommend_list.append(
      df_recommend,
      ignore_index=True
  )


df_recommend_list

,user_id,item_id,score,rank
0,1,285,5.037234,1
1,1,423,3.889698,2
2,1,408,3.800589,3
3,1,655,3.713240,4
4,1,474,3.188664,5
...,...,...,...,...
9425,943,288,2.826179,6
9426,943,273,2.770847,7
9427,943,17,2.737602,8
9428,943,550,2.733348,9
